In [1]:
import sys
sys.executable

'/media/rehan/yaws/ws_new/OpenKE/py/bin/python3'

In [180]:
import torch
import os
import sys
sys.path.append('../')
from models import *
import config
import numpy as np

In [181]:
from collections import defaultdict

In [182]:
in_path = '../benchmarks/ECB/'
# in_path = '../benchmarks/FB15K237_names/'
with open( in_path + 'entity2id.txt') as nf:
    lines = [line.split('\t') for line in nf.readlines()[1:]]
    ent2id = {line[0]:int(line[1]) for line in lines}
ent2id   

{'10_3ecb_111_126': 0,
 '10_3ecb_130_133': 1,
 '10_4ecb_1027_1034': 2,
 '10_4ecb_1061_1062': 3,
 '10_4ecb_1075_1079': 4,
 '10_4ecb_1995_2002': 5,
 '10_4ecb_2043_2046': 6,
 '10_4ecb_489_494': 7,
 '10_4ecb_507_511': 8,
 '10_4ecb_529_533': 9,
 '10_4ecb_550_557': 10,
 '10_4ecb_562_565': 11,
 '10_4ecb_960_969': 12,
 '10_4ecb_986_989': 13,
 '10_5ecb_875_877': 14,
 '10_5ecb_878_881': 15,
 '10_6ecb_105_108': 16,
 '10_6ecb_329_337': 17,
 '10_6ecb_341_344': 18,
 '10_6ecb_351_358': 19,
 '10_6ecb_86_101': 20,
 '10_6ecb_86_92': 21,
 '10_7ecb_1167_1174': 22,
 '10_7ecb_1188_1193': 23,
 '10_7ecb_1212_1213': 24,
 '10_7ecb_1226_1230': 25,
 '10_7ecb_732_737': 26,
 '10_7ecb_750_754': 27,
 '10_7ecb_772_776': 28,
 '10_7ecb_793_800': 29,
 '10_7ecb_805_808': 30,
 '11_1ecb_22_30': 31,
 '11_1ecb_34_46': 32,
 '11_1ecb_53_59': 33,
 '11_2ecb_1055_1059': 34,
 '11_2ecb_1063_1065': 35,
 '11_2ecb_1080_1085': 36,
 '11_2ecb_1086_1090': 37,
 '11_2ecb_1238_1241': 38,
 '11_2ecb_1246_1250': 39,
 '11_2ecb_1272_1279': 40,
 '1

In [185]:
with open('../benchmarks/ECB/triples/eve_map.csv') as em:
    events = set([line.split(',')[0] for line in em.readlines()])

In [186]:
event_ids_dict = {line[0]:int(line[1]) for line in lines if line[0] in events}
event_ids = event_ids_dict.values()

In [188]:
len(event_ids)

1975

In [189]:
event2ind = {eve:i for i, eve in enumerate(event_ids_dict.keys())}
ind2event = {i:eve for eve, i in event2ind.items()}

In [190]:
con2 = config.Config()
con2.set_in_path(in_path)
con2.init()

In [191]:
te = TransE(con2)
te.load_state_dict(torch.load('../result/TransE.ckpt'))

/media/rehan/yaws/ws_new/OpenKE/py/lib/python3.6/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
../models/TransE.py:23: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.ent_embeddings.weight.data)
../models/TransE.py:24: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.rel_embeddings.weight.data)
../models/TransE.py:26: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.non_linear_weights.weight.data)


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [192]:
event_embeddings = te.ent_embeddings(torch.tensor(np.array(list(event_ids)))).cpu().data

In [193]:
event_embeddings

tensor([[-0.0008,  0.0203, -0.0139,  ...,  0.0314,  0.0291,  0.0214],
        [ 0.0160,  0.0378,  0.0165,  ...,  0.0082,  0.0261, -0.0168],
        [ 0.0064, -0.0149,  0.0137,  ...,  0.0119, -0.0008,  0.0162],
        ...,
        [ 0.0086, -0.0353,  0.0226,  ...,  0.0099,  0.0114,  0.0133],
        [-0.0163,  0.0335, -0.0034,  ...,  0.0357,  0.0217,  0.0186],
        [-0.0263,  0.0295,  0.0270,  ...,  0.0140, -0.0275,  0.0271]])

In [194]:
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix

In [197]:
event_embeddings_normed = np.array(event_embeddings)
event_embeddings_normed = event_embeddings_normed/(np.linalg.norm(event_embeddings_normed, axis=1).reshape((-1,1)))
eve_sim_mat = event_embeddings_normed.dot(event_embeddings_normed.T)
eve_adj_mat = csr_matrix(eve_sim_mat > 0.45)
eve_adj_mat

<1975x1975 sparse matrix of type '<class 'numpy.bool_'>'
	with 2067 stored elements in Compressed Sparse Row format>

In [198]:
def get_connected_components(adjacency_mat):
    n_components, labels = connected_components(csgraph=adjacency_mat, directed=False, return_labels=True)
    clusters = defaultdict(list)
    for id_, lab in enumerate(labels):
        clusters[lab] += [id_]
    return clusters.values()

In [199]:
eve_clusters = get_connected_components(eve_adj_mat)
eve_clusters_big = [clus for clus in eve_clusters if len(clus) >= 2]

In [200]:
eve_clusters_big

[[23, 165, 1040, 1454],
 [97, 448, 449, 590, 591, 638],
 [125, 1409],
 [144, 356, 845, 1065, 1531, 1827],
 [251, 1106],
 [263, 770],
 [267, 626, 1047],
 [320, 949],
 [427, 644, 1953],
 [489, 659],
 [523, 798],
 [555, 1139],
 [569, 1221],
 [579, 1623],
 [583, 1161, 1937],
 [586, 1146],
 [592, 1585],
 [614, 632],
 [656, 1622],
 [689, 711],
 [758, 1110, 1402],
 [927, 1854],
 [1123, 1165],
 [1183, 1561],
 [1215, 1696],
 [1330, 1886],
 [1564, 1815],
 [1587, 1588],
 [1620, 1659, 1661, 1677],
 [1927, 1972]]

In [163]:
# event_ids_dict['events/3248']

KeyError: 'events/3248'

In [164]:
len(event_ids)

123

In [201]:
with open('../benchmarks/ECB/triples/eve_map.csv') as ef:
    eve_dict = {line.strip().split(',')[1]:line.strip().split(',')[0] for line in ef.readlines()}

In [205]:
eve_clus_dict = {}
clus_num = 0
for clus in list(eve_clusters):
    for eve_ind in clus:
        eve_clus_dict[eve_dict[ind2event[eve_ind]]] = str(clus_num)
    clus_num+=1

In [204]:
with open('event_clusters.txt', 'w') as ef:
    ef.write('\n'.join([','.join(item) for item in eve_clus_dict.items()]))
with open('event_win_clus.txt', 'w') as wf:
    wf.write('\n'.join([','.join([eve, str(i)]) for i, eve in enumerate(eve_clus_dict.keys())]))